In [9]:
import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_glpk ()
    helper.install_idaes()
    helper.install_ipopt()
!pip install requests folium
!pip install openrouteservice

In [10]:
import pandas as pd

df_clients = pd.read_csv('./content/caso3/clients.csv')
df_depots = pd.read_csv('./content/caso3/depots.csv')
df_vehicles = pd.read_csv('./content/caso3/vehicles.csv')

In [11]:
df_depots['DepotID'] = df_depots['DepotID'].astype(str)
df_depots['ClientID'] = df_clients['ClientID'].astype(str)
df_vehicles['VehicleName'] = [f'V{i+1}' for i in range(len(df_vehicles))]


coord_depots = list(zip(df_depots['Longitude'], df_depots['Latitude']))

coord_clients = list(zip(df_clients['Longitude'], df_clients['Latitude']))

lugares = coord_depots + coord_clients

In [12]:
precio_km = 3032.1

# Leer el CSV
df = pd.read_csv('distances_all.csv')

# Inicializar ambos diccionarios
distance = {}
cost     = {}

# Recorrer una única vez (usando zip para eficiencia)
for ori, dst, d in zip(df['origen'], df['destino'], df['distancia_km']):
    key = (ori, dst)
    distance[key] = d
    cost[key]     = d * precio_km

# Verificar
print(distance[('CD1','CD2')], cost[('CD1','CD2')])

32.99 100028.979


In [13]:
depot_capacity = {}
for depot, d in zip(df_depots['DepotID'], df_depots['Capacity']):
    depot_capacity[f'CD{depot}'] = d

print(depot_capacity)

demand = {}
for client, c in zip(df_clients['ClientID'], df_clients['Demand']):
    demand[f'C{client}'] = c

print(demand)

vehicle_capacity={}
for vehicle, c in zip(df_vehicles['VehicleName'], df_vehicles['Capacity']):
    vehicle_capacity[vehicle] = c

print(vehicle_capacity)

vehicle_range={}
for vehicle, r in zip(df_vehicles['VehicleName'], df_vehicles['Range']):
    vehicle_range[vehicle] = r

print(vehicle_range)


{'CD1': 11, 'CD2': 90, 'CD3': 130, 'CD4': 145, 'CD5': 260, 'CD6': 180, 'CD7': 720, 'CD8': 55, 'CD9': 70, 'CD10': 75, 'CD11': 90, 'CD12': 270}
{'C1': 12, 'C2': 12, 'C3': 12, 'C4': 12, 'C5': 12, 'C6': 12, 'C7': 12, 'C8': 12, 'C9': 12, 'C10': 12, 'C11': 12, 'C12': 12, 'C13': 8, 'C14': 12, 'C15': 12, 'C16': 12, 'C17': 12, 'C18': 12, 'C19': 12, 'C20': 12, 'C21': 12, 'C22': 12, 'C23': 12, 'C24': 12, 'C25': 12, 'C26': 12, 'C27': 12, 'C28': 12, 'C29': 12, 'C30': 12, 'C31': 11, 'C32': 12, 'C33': 12, 'C34': 12, 'C35': 12, 'C36': 12, 'C37': 12, 'C38': 12, 'C39': 12, 'C40': 12, 'C41': 12, 'C42': 12, 'C43': 12, 'C44': 12, 'C45': 12, 'C46': 12, 'C47': 12, 'C48': 12, 'C49': 12, 'C50': 12, 'C51': 12, 'C52': 12, 'C53': 12, 'C54': 12, 'C55': 12, 'C56': 12, 'C57': 9, 'C58': 12, 'C59': 12, 'C60': 12, 'C61': 12, 'C62': 12, 'C63': 12, 'C64': 12, 'C65': 12, 'C66': 12, 'C67': 12, 'C68': 12, 'C69': 12, 'C70': 12, 'C71': 12, 'C72': 12, 'C73': 12, 'C74': 12, 'C75': 12, 'C76': 12, 'C77': 12, 'C78': 12, 'C79': 12,

In [90]:
# 1) Preparamos listas y diccionarios de distancias
depot_ids   = list(df_depots['DepotID'].astype(str))
client_ids  = list(df_clients['ClientID'].astype(str))

depot_labels  = [f"CD{d}" for d in depot_ids]
client_labels = [f"C{c}"  for c in client_ids]

# 2) Calculamos cuántos clientes debe atender cada depósito
n_clients = len(client_labels)
n_depots  = len(depot_labels)

# 3) Inicializamos estructuras
clients_per_depot = {d: [] for d in depot_labels}
remaining_capacity = depot_capacity.copy()  # capacidad restante real
max_clients_per_depot = 9
max_distance = 60

# 4) Precomputamos los depósitos más cercanos a cada cliente (dentro de rango)
nearest_depots = {
    c: sorted(
        depot_labels,
        key=lambda d: distance[(d, c)] if distance[(d, c)] <= max_distance else float('inf')
    )
    for c in client_labels
}

# 5) Asignación greedy respetando capacidad, distancia y cantidad máxima de clientes
unassigned_clients = []
for c in client_labels:
    demand_c = demand[c]
    assigned = False
    for d in nearest_depots[c]:
        if remaining_capacity[d] >= demand_c and distance[(d, c)] <= max_distance and len(clients_per_depot[d]) < max_clients_per_depot:
            clients_per_depot[d].append(c)
            remaining_capacity[d] -= demand_c
            assigned = True
            break
    if not assigned:
        unassigned_clients.append(c)
        print(f"No se pudo asignar al cliente {c}: demanda {demand_c}, fuera del rango de distancia o límite de clientes alcanzado.")

# 6) Segunda pasada: asigna clientes no asignados al depósito con menos carga (sin restricciones de distancia)
# 6) Segunda pasada: asigna clientes no asignados al depósito con menos carga (sin restricciones de distancia)
for c in unassigned_clients:
    demand_c = demand[c]
    
    # Ordena los depósitos por la demanda total de los clientes que ya tienen asignados
    sorted_depots = sorted(
        depot_labels,
        key=lambda d: sum(demand[cli] for cli in clients_per_depot[d])  # Ordena según la demanda total
    )
    for d in sorted_depots:
        if len(clients_per_depot[d]) == 0:  # Si el depósito no tiene clientes
            clients_per_depot[d].append(c)
            remaining_capacity[d] -= demand_c 
            print(f"Cliente {c} (demanda {demand_c}) asignado forzadamente a {d}")
            break
        elif remaining_capacity[d] >= demand_c:
            clients_per_depot[d].append(c)
            remaining_capacity[d] -= demand_c  
            print(f"Cliente {c} asignado a {d} con demanda {demand_c}")
            break
    else:
        print(f"Cliente {c} NO pudo ser asignado")

for d, clist in clients_per_depot.items():
    total_demanda = sum(demand[c] for c in clist)
    print(f"{d} atiende a {len(clist)} clientes (demanda total: {total_demanda}): {clist}")

No se pudo asignar al cliente C85: demanda 12, fuera del rango de distancia o límite de clientes alcanzado.
No se pudo asignar al cliente C86: demanda 12, fuera del rango de distancia o límite de clientes alcanzado.
No se pudo asignar al cliente C88: demanda 12, fuera del rango de distancia o límite de clientes alcanzado.
No se pudo asignar al cliente C89: demanda 12, fuera del rango de distancia o límite de clientes alcanzado.
No se pudo asignar al cliente C90: demanda 12, fuera del rango de distancia o límite de clientes alcanzado.
Cliente C85 (demanda 12) asignado forzadamente a CD1
Cliente C86 asignado a CD6 con demanda 12
Cliente C88 asignado a CD3 con demanda 12
Cliente C89 asignado a CD4 con demanda 12
Cliente C90 asignado a CD5 con demanda 12
CD1 atiende a 1 clientes (demanda total: 12): ['C85']
CD2 atiende a 8 clientes (demanda total: 89): ['C70', 'C72', 'C73', 'C75', 'C76', 'C79', 'C80', 'C83']
CD3 atiende a 10 clientes (demanda total: 119): ['C5', 'C6', 'C31', 'C74', 'C77', 

In [91]:
vehicles_per_depot = {d: [] for d in depot_labels}
available_vehicles = vehicle_labels.copy()
assigned_vehicles = set()

for d in depot_labels:
    # Calcular demanda total del depósito
    client_list = clients_per_depot[d]
    if not client_list:
        continue  # Saltar si no hay clientes asignados

    total_demand = sum(demand[c] for c in client_list)
    cap_sum = 0

    for v in vehicle_labels:
        if v not in assigned_vehicles:
            vehicles_per_depot[d].append(v)
            cap_sum += vehicle_capacity[v]
            assigned_vehicles.add(v)

        if cap_sum >= total_demand:
            break

    if cap_sum < total_demand:
        print(f"⚠️ Depósito {d} tiene demanda {total_demand} pero vehículos asignados cubren solo {cap_sum}")

# Mostrar resultado
for d, vs in vehicles_per_depot.items():
    if vs:  # Solo mostrar si hay vehículos asignados
        print(f"{d} tiene asignados {len(vs)} vehículos: {vs}")


CD1 tiene asignados 1 vehículos: ['V1']
CD2 tiene asignados 1 vehículos: ['V2']
CD3 tiene asignados 2 vehículos: ['V3', 'V4']
CD4 tiene asignados 2 vehículos: ['V5', 'V6']
CD5 tiene asignados 1 vehículos: ['V7']
CD6 tiene asignados 2 vehículos: ['V8', 'V9']
CD7 tiene asignados 2 vehículos: ['V10', 'V11']
CD8 tiene asignados 1 vehículos: ['V12']
CD9 tiene asignados 1 vehículos: ['V13']
CD10 tiene asignados 1 vehículos: ['V14']
CD11 tiene asignados 1 vehículos: ['V15']
CD12 tiene asignados 5 vehículos: ['V16', 'V17', 'V18', 'V19', 'V20']


In [92]:
# Etiquetas
depot_ids    = [str(d) for d in df_depots['DepotID']]
depot_labels = [f"CD{d}" for d in depot_ids]
client_ids   = [str(c) for c in df_clients['ClientID']]
client_labels= [f"C{c}"  for c in client_ids]
vehicle_list = df_vehicles['VehicleName'].tolist()

# Mapeo cliente → depósito fijo
"""
assign_client = {}
for d, clist in clients_per_depot.items():
    for c in clist:
        assign_client[f"C{c}"] = f"CD{d}"
"""
assign_client = {
    c: d
    for d, clist in clients_per_depot.items()
    for c in clist
}

In [ ]:
assign_vehicle = {} 
available_vehicles = set(vehicle_list)
assigned_vehicles  = set()
vehicles_per_depot = {d: [] for d in depot_labels}


for d in depot_labels:
    client_list = clients_per_depot[d]
    if not client_list:
        continue  

    total_demand = sum(demand[c] for c in client_list)
    total_capacity = 0

    for v in vehicle_list:
        if v not in assigned_vehicles:
            vehicle_cap = vehicle_capacity[v]
            assign_vehicle[v] = d
            vehicles_per_depot[d].append(v)
            assigned_vehicles.add(v)
            total_capacity += vehicle_cap

        if total_capacity >= total_demand:
            break

    if total_capacity < total_demand:
        print(f"⚠️ {d} necesita {total_demand} unidades, pero los vehículos asignados cubren solo {total_capacity}")

for d, vlist in vehicles_per_depot.items():
    if vlist:
        print(f"{d} recibe vehículos: {vlist}")


CD1 recibe vehículos: ['V1']
CD2 recibe vehículos: ['V2']
CD3 recibe vehículos: ['V3', 'V4']
CD4 recibe vehículos: ['V5', 'V6']
CD5 recibe vehículos: ['V7']
CD6 recibe vehículos: ['V8', 'V9']
CD7 recibe vehículos: ['V10', 'V11']
CD8 recibe vehículos: ['V12']
CD9 recibe vehículos: ['V13']
CD10 recibe vehículos: ['V14']
CD11 recibe vehículos: ['V15']
CD12 recibe vehículos: ['V16', 'V17', 'V18', 'V19', 'V20']


In [94]:
print("Claves de assign_client:", sorted(assign_client.keys())[:10], "…", len(assign_client))


Claves de assign_client: ['C1', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18'] … 90


In [95]:
assign_client

{'C85': 'CD1',
 'C70': 'CD2',
 'C72': 'CD2',
 'C73': 'CD2',
 'C75': 'CD2',
 'C76': 'CD2',
 'C79': 'CD2',
 'C80': 'CD2',
 'C83': 'CD2',
 'C5': 'CD3',
 'C6': 'CD3',
 'C31': 'CD3',
 'C74': 'CD3',
 'C77': 'CD3',
 'C78': 'CD3',
 'C81': 'CD3',
 'C82': 'CD3',
 'C84': 'CD3',
 'C88': 'CD3',
 'C2': 'CD4',
 'C4': 'CD4',
 'C7': 'CD4',
 'C20': 'CD4',
 'C41': 'CD4',
 'C47': 'CD4',
 'C58': 'CD4',
 'C63': 'CD4',
 'C66': 'CD4',
 'C89': 'CD4',
 'C3': 'CD5',
 'C16': 'CD5',
 'C17': 'CD5',
 'C18': 'CD5',
 'C19': 'CD5',
 'C28': 'CD5',
 'C42': 'CD5',
 'C45': 'CD5',
 'C46': 'CD5',
 'C90': 'CD5',
 'C1': 'CD6',
 'C11': 'CD6',
 'C13': 'CD6',
 'C26': 'CD6',
 'C30': 'CD6',
 'C32': 'CD6',
 'C33': 'CD6',
 'C48': 'CD6',
 'C55': 'CD6',
 'C86': 'CD6',
 'C10': 'CD7',
 'C12': 'CD7',
 'C14': 'CD7',
 'C15': 'CD7',
 'C23': 'CD7',
 'C24': 'CD7',
 'C34': 'CD7',
 'C35': 'CD7',
 'C39': 'CD7',
 'C44': 'CD8',
 'C62': 'CD8',
 'C69': 'CD8',
 'C71': 'CD8',
 'C9': 'CD9',
 'C21': 'CD9',
 'C29': 'CD9',
 'C49': 'CD9',
 'C61': 'CD9',
 'C

In [96]:
assign_vehicle

{'V1': 'CD1',
 'V2': 'CD2',
 'V3': 'CD3',
 'V4': 'CD3',
 'V5': 'CD4',
 'V6': 'CD4',
 'V7': 'CD5',
 'V8': 'CD6',
 'V9': 'CD6',
 'V10': 'CD7',
 'V11': 'CD7',
 'V12': 'CD8',
 'V13': 'CD9',
 'V14': 'CD10',
 'V15': 'CD11',
 'V16': 'CD12',
 'V17': 'CD12',
 'V18': 'CD12',
 'V19': 'CD12',
 'V20': 'CD12'}

In [97]:
from pyomo.environ import (ConcreteModel, Set, Param, Var, Objective, Constraint,
                           Binary, NonNegativeReals, minimize, value)
from pyomo.opt import SolverFactory

model = ConcreteModel()

# 2.1) Conjuntos
model.D = Set(initialize=depot_labels)
model.C = Set(initialize=client_labels)
model.V = Set(initialize=vehicle_list)
model.N = model.D | model.C

# 2.2) Parámetros de costo y distancia (ya def. en tu script)
model.cost     = Param(model.N, model.N, initialize=lambda m,i,j: cost[(i,j)],     within=NonNegativeReals)
model.distance = Param(model.N, model.N, initialize=lambda m,i,j: distance[(i,j)], within=NonNegativeReals)

# 2.3) Capacidades y rangos
model.depot_capacity   = Param(model.D, initialize=lambda m,d: depot_capacity[d])
model.vehicle_capacity = Param(model.V, initialize=lambda m,v: vehicle_capacity[v])
model.vehicle_range    = Param(model.V, initialize=lambda m,v: vehicle_range[v])
model.demand           = Param(model.C, initialize=lambda m,c: demand[c])

# 2.4) Variables
# y[v,d]=1 si v parte del depósito d
model.y = Var(model.V, model.D, domain=Binary)
# x[v,i,j]=1 si v viaja de i a j
model.x = Var(model.V, model.N, model.N, domain=Binary)
# MTZ
model.u = Var(model.V, model.C, domain=NonNegativeReals)

# 2.5) Objetivo: minimizar costo total
def obj_rule(m):
    return sum(m.cost[i,j] * m.x[v,i,j]
               for v in m.V for i in m.N for j in m.N if i!=j)
model.obj = Objective(rule=obj_rule, sense=minimize)

# ──────────────
# 2.6) Restricciones
# a) cada vehículo va a lo sumo a un depósito  
def one_depot_per_vehicle(m, v):
    return sum(m.y[v,d] for d in m.D) <= 1
model.one_depot = Constraint(model.V, rule=one_depot_per_vehicle)

# b) si v está asignado a d, sale exactamente una vez de d; si no, no sale  
def start_route(m, v, d):
    return sum(m.x[v,d,j] for j in m.N if j!=d) == m.y[v,d]
model.start_route = Constraint(model.V, model.D, rule=start_route)

# c) igual para el regreso  
def return_route(m, v, d):
    return sum(m.x[v,i,d] for i in m.N if i!=d) == m.y[v,d]
model.return_route = Constraint(model.V, model.D, rule=return_route)

# d) conservación de flujo en clientes  
def flow_cons(m, v, i):
    if i in m.C:
        return ( sum(m.x[v,j,i] for j in m.N if j!=i)
               == sum(m.x[v,i,j] for j in m.N if j!=i) )
    return Constraint.Skip
model.flow_cons = Constraint(model.V, model.N, rule=flow_cons)

def only_own_clients(m, v, i, j):
    # 1) Depósito → Depósito: no imponemos nada
    if i in m.D and j in m.D:
        return Constraint.Skip

    # 2) Arco que termina en cliente j: 
    #    solo si v partió de su depósito asignado
    if j in m.C:
        d_j = assign_client[j]
        return m.x[v, i, j] <= m.y[v, d_j]

    # 3) Arco que sale de cliente i → depósito j:
    if i in m.C and j in m.D:
        d_i = assign_client[i]
        return m.x[v, i, j] <= m.y[v, d_i]

    # 4) Cliente → Cliente: MTZ + flujo se encargan, aquí skip
    return Constraint.Skip

model.only_own = Constraint(model.V, model.N, model.N, rule=only_own_clients)


# f) cada cliente lo visita exactamente un vehículo
def each_client_once(m, c):
    return sum(m.x[v,j,c] for v in m.V for j in m.N if j!=c) == 1
model.visit_once = Constraint(model.C, rule=each_client_once)

# g) capacidad vehículo (si no está asignado, y[v,d]==0 → x=0 → carga=0)  
def vehicle_cap(m, v):
    return sum( m.demand[i] * sum(m.x[v,j,i] for j in m.N if j!=i)
               for i in m.C ) <= sum(m.y[v,d]*m.vehicle_capacity[v] for d in m.D)
model.veh_cap = Constraint(model.V, rule=vehicle_cap)

# h) capacidad depósito: suma de demandas servidas desde d ≤ cap[d]  
# (opcional) pre-extrae las listas para que queden como depot_clients
depot_clients = clients_per_depot

def depot_cap(m, d):
    if not depot_clients[d]:  # si no hay clientes asignados al depósito
        return Constraint.Skip

    return sum(
        m.demand[i] * sum(m.x[v, j, i] for j in m.N if j != i)
        for v in m.V
        for i in depot_clients[d]
    ) <= m.depot_capacity[d]

model.dep_cap = Constraint(model.D, rule=depot_cap)


# i) rango vehículo  
def veh_range(m, v):
    return sum(m.distance[i,j]*m.x[v,i,j] for i in m.N for j in m.N if i!=j) \
           <= sum(m.y[v,d]*m.vehicle_range[v] for d in m.D)
model.veh_range = Constraint(model.V, rule=veh_range)

# j) MTZ subtours (igual que antes)
def mtz_rule(m, v, i, j):
    if i!=j:
        return (m.u[v,i] - m.u[v,j]
               + m.vehicle_capacity[v]*m.x[v,i,j]
               <= m.vehicle_capacity[v] - m.demand[j])
    return Constraint.Skip
model.mtz     = Constraint(model.V, model.C, model.C, rule=mtz_rule)
model.mtz_bnd = Constraint(model.V, model.C,
                           rule=lambda m,v,i: (m.demand[i], m.u[v,i], m.vehicle_capacity[v]))

# k) satisfacción de demanda parcial (MTZ auxiliar)
def demand_sat(m, v, i):
    return m.u[v,i] >= m.demand[i] * sum(m.x[v,j,i] for j in m.N if j!=i)
model.demand_sat = Constraint(model.V, model.C, rule=demand_sat)

# l) Al menos el 80% de los vehículos deben salir
def at_least_80_percent_vehicles(m):
    total_vehicles = len(m.V)  # Total de vehículos
    required_vehicles = int(0.8 * total_vehicles)  # 80% de vehículos
    return sum(sum(m.y[v,d] for d in m.D) for v in m.V) >= required_vehicles
model.at_least_80_percent_vehicles = Constraint(rule=at_least_80_percent_vehicles)

# Restricción: solo permitir y[v,d] si está en assign_vehicle[d]
def respect_vehicle_assignment(m, v, d):
    if v not in assign_vehicle.get(d, []):
        return m.y[v, d] == 0  # No permitido
    return Constraint.Skip

model.vehicle_assign_limit = Constraint(model.V, model.D, rule=respect_vehicle_assignment)




In [98]:
def print_routes(model):
    for v in model.V:
        print(f"\n>>> Ruta del vehículo {v}:")
        
        # 1. Encontrar el depósito asignado al vehículo y su capacidad
        depot = None
        depot_cap = 0
        for d in model.D:
            if value(model.y[v, d]) > 0.5:
                depot = d
                depot_cap = depot_capacity[d]
                break
        
        if depot is None:
            print("Vehículo no asignado a ningún depósito.")
            continue
        
        print(f" - Depósito de salida: {depot} (Capacidad: {depot_cap})")
        print(f" - Capacidad del vehículo: {vehicle_capacity[v]}")
        
        # 2. Reconstruir la ruta y calcular carga
        current_node = depot
        route = [current_node]
        visited = set()
        total_load = 0
        deliveries = {}  # Diccionario para guardar entregas entre nodos
        
        while True:
            next_node = None
            for j in model.N:
                if j != current_node and value(model.x[v, current_node, j]) > 0.5:
                    next_node = j
                    break
            
            if next_node is None or next_node in visited:
                break
            
            # Registrar entrega si es a un cliente
            if next_node in model.C:
                delivery = demand[next_node]
                total_load += delivery
                deliveries[(current_node, next_node)] = delivery
            
            route.append(next_node)
            visited.add(next_node)
            current_node = next_node
        
        # 3. Imprimir resultados
        if len(route) > 1:
            print(f" - Ruta: {' → '.join(str(node) for node in route)}")
            print(f" - Carga total transportada: {total_load}/{vehicle_capacity[v]}")
            
            # Imprimir entregas entre pares de nodos
            print(" - Entregas entre nodos:")
            for (i, j), qty in deliveries.items():
                print(f"    De {i} → {j}: Entrega {qty} unidades")
        else:
            print("Vehículo no utilizado.")


In [99]:
from amplpy import modules

solver_name = "gurobi" 
solver = SolverFactory(solver_name+"nl", executable=modules.find(solver_name), solve_io="nl")
solver.options['TimeLimit'] = 500
solver.options['MIPGap']   = 0.01

res = solver.solve(model, load_solutions=True)

from pyomo.opt import TerminationCondition
if res.solver.termination_condition not in (
    TerminationCondition.optimal,
    TerminationCondition.feasible):
    raise RuntimeError(f"No factible/óptimo: {res.solver.termination_condition}")

# Ahora, cada y[v,d] te indica si el vehículo v está en el centro d
# y las x[v,i,j] la ruta que sigue desde d → clientes(asignados) → d.

# Imprime las rutas usando tu función actualizada:
print_routes(model)


python(18961) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(19079) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


model.name="unknown";
    - termination condition: infeasible
    - message from solver: Gurobi 12.0.1\x3a infeasible problem; 0 simplex
      iterations


RuntimeError: No factible/óptimo: infeasible